### Import files

In [2]:
import numpy as np
import pandas as pd
import os, statistics

path = "C:\\Users\pault\OneDrive\Documenten\GitHub\input" # pauls path
#add your path here
os.chdir(path)

train = pd.read_csv('train.tsv', delimiter='\t', encoding='utf-8')
print("train size:", train.size)

test = pd.read_csv('test.tsv', delimiter='\t', encoding='utf-8')
print("test size:", test.size)

train size: 11860280
test size: 4853513


### Small test space

In [115]:

print(train.size)
train.head(5)
train.get_value(1, "price")
a = statistics.mean([1, 4, 5, 6, 4, 3, 2])
print(a)

11860280
3.5714285714285716


### Variance
To take a first look at the data, we can sort the data on a column's values and watch what happens with the variance. 

In [129]:
sample = train.loc[0:10000]

# sorts a column and returns a dictionary with a list of indexes that have the same text/number in that column. 
def sort_index_by_column(data, column):
    dct = {'nan': []}
    for index, row in data.iterrows():
        if row[column] == row[column]:
            if row[column] in dct: 
                dct[row[column]].append(index)
            else: 
                dct[row[column]] = [index]
        else:
            dct["nan"].append(index)
    return dct

#deletes all the keys that have a list with only one value 
def delete_einzelgangers(dictionary):
    new_dict = {}
    for key in dictionary:
        if len(dictionary[key]) != 1:
               new_dict[key] = dictionary[key]
    return new_dict

# for every list, replace all the indexes with prices
def replace_index_with_price(dicti, data):
    new_dict = {}
    for key in dicti: 
        lst = []
        for ind in dicti[key]: 
            lst.append(data.get_value(ind, "price"))
        new_dict[key] = lst
    return new_dict

# calculates various things for every key
def mean_var_amount_per_dict_key(dicti):
    mean_dict = {}
    for key in dicti: 
        mean = statistics.mean(dicti[key])
        variance = statistics.variance(dicti[key])
        amount = len(dicti[key])
        mean_dict[key] = [mean, variance, int(amount)]
    return mean_dict
    
sort_brand_name_ind = delete_einzelgangers(sort_index_by_column(sample, "brand_name"))
#print(sort_brand_name_ind)
sort_brand_name_price = replace_index_with_price(sort_brand_name_ind, sample)
#print(sort_brand_name_price)
sort_brand_name_mva = mean_var_amount_per_dict_key(sort_brand_name_price)

for key in sort_brand_name_mva:
    print(key + " - mean:", sort_brand_name_mva[key][0], "variance:", 
    sort_brand_name_mva[key][1], "amount:", sort_brand_name_mva[key][2])

nan - mean: 20.2133302042 variance: 552.456474846 amount: 4261
Razer - mean: 38.5 variance: 364.5 amount: 2
Target - mean: 17.25 variance: 345.113636364 amount: 12
Acacia Swimwear - mean: 79.2 variance: 3145.2 amount: 5
Nike - mean: 29.6942857143 variance: 826.447818256 amount: 350
Smashbox - mean: 16.3333333333 variance: 41.2380952381 amount: 15
Victoria's Secret - mean: 23.2267080745 variance: 265.228817167 amount: 322
rue - mean: 14.1111111111 variance: 38.6111111111 amount: 9
UGG Australia - mean: 49.0689655172 variance: 857.780788177 amount: 29
Tarte - mean: 23.5 variance: 124.542857143 amount: 36
Wet n Wild - mean: 7.44444444444 variance: 15.7777777778 amount: 9
Too Faced - mean: 18.9038461538 variance: 58.1278280543 amount: 52
Anthropologie - mean: 23.6666666667 variance: 12.3333333333 amount: 3
Torrid - mean: 19.9230769231 variance: 51.7435897436 amount: 13
Samsung - mean: 86.3214285714 variance: 15915.1891534 amount: 28
FOREVER 21 - mean: 11.75 variance: 30.7686915888 amount: 

Hollister Co. - mean: 11.5 variance: 55.0 amount: 4
ASICS - mean: 31.3333333333 variance: 479.066666667 amount: 6
Simply Southern - mean: 9.0 variance: 50.0 amount: 2
Fisher-Price - mean: 29.2857142857 variance: 283.904761905 amount: 7
Microsoft - mean: 15.0 variance: 193.5 amount: 5
Jeffree Star Cosmetics - mean: 17.0 variance: 19.0 amount: 3
Louis Vuitton - mean: 188.647058824 variance: 35524.9926471 amount: 17
Bebe - mean: 26.4285714286 variance: 291.619047619 amount: 7
Aeropostale - mean: 14.125 variance: 107.716666667 amount: 16
Vintage - mean: 22.2307692308 variance: 159.358974359 amount: 13
Lilly Pulitzer - mean: 45.1578947368 variance: 406.584795322 amount: 19
LG - mean: 30.125 variance: 120.125 amount: 8
Thrasher Magazine - mean: 19.2 variance: 15.7 amount: 5
Moose Toys - mean: 16.0 variance: 160.666666667 amount: 4
Home Interiors - mean: 15.3333333333 variance: 162.333333333 amount: 3
LEGO - mean: 20.6666666667 variance: 215.5 amount: 9
Carter's - mean: 16.375 variance: 120.3